In [1]:
# Cell 1 - Setup and Data Loading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("🧹 XO Project - Data Cleaning Phase")
print("="*60)
print("Objective: Clean and prepare exoplanet data for habitability analysis")
print("="*60)

# Load the raw data (using knowledge from notebook 01)
with open('../data/raw/exoplanet_data.csv', 'r') as f:
    lines = f.readlines()

# Find header line containing actual column names
header_line_index = None
for i, line in enumerate(lines):
    if 'pl_name' in line and not line.strip().startswith('#'):
        header_line_index = i
        print(f"Found header at line {i + 1}")
        break

# Load data starting from header
df_raw = pd.read_csv('../data/raw/exoplanet_data.csv',
                     skiprows=header_line_index,
                     sep=',',
                     on_bad_lines='skip',
                     low_memory=False)

print(f"Raw dataset loaded: {df_raw.shape[0]:,} planets, {df_raw.shape[1]} features")
print(f"Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

🧹 XO Project - Data Cleaning Phase
Objective: Clean and prepare exoplanet data for habitability analysis
Found header at line 97
Raw dataset loaded: 38,779 planets, 92 features
Memory usage: 68.9 MB


In [2]:
# Cell 2 - Data Quality Assessment
print("\nData Quality Assessment:")
print("="*50)

# Define core habitability features based on our exploration
core_features = {
    'pl_name': 'Planet Name',
    'hostname': 'Host Star Name',
    'pl_rade': 'Planet Radius (Earth radii)',
    'pl_bmasse': 'Planet Mass (Earth masses)',
    'pl_orbsmax': 'Orbital Distance (AU)',
    'st_teff': 'Stellar Temperature (K)',
    'st_mass': 'Stellar Mass (Solar masses)',
    'pl_eqt': 'Equilibrium Temperature (K)',
    'discoverymethod': 'Discovery Method',
    'disc_year': 'Discovery Year'
}

# Check availability and completeness
available_features = {}
for feature, description in core_features.items():
    if feature in df_raw.columns:
        non_null = df_raw[feature].count()
        total = len(df_raw)
        completeness = (non_null / total) * 100
        available_features[feature] = completeness
        print(f"✓ {feature:12} | {description:35} | {completeness:5.1f}% ({non_null:,})")
    else:
        print(f"✗ {feature:12} | {description:35} | Not found")

# Identify minimum viable dataset
min_required = ['pl_name', 'pl_rade', 'pl_orbsmax', 'st_teff']
viable_subset = df_raw.dropna(subset=min_required)
print(f"\nPlanets with minimum required data: {len(viable_subset):,} ({len(viable_subset)/len(df_raw)*100:.1f}%)")


Data Quality Assessment:
✓ pl_name      | Planet Name                         | 100.0% (38,779)
✓ hostname     | Host Star Name                      | 100.0% (38,779)
✓ pl_rade      | Planet Radius (Earth radii)         |  69.1% (26,789)
✓ pl_bmasse    | Planet Mass (Earth masses)          |  17.5% (6,768)
✓ pl_orbsmax   | Orbital Distance (AU)               |  56.2% (21,779)
✓ st_teff      | Stellar Temperature (K)             |  91.4% (35,435)
✓ st_mass      | Stellar Mass (Solar masses)         |  84.6% (32,797)
✓ pl_eqt       | Equilibrium Temperature (K)         |  43.7% (16,951)
✓ discoverymethod | Discovery Method                    | 100.0% (38,779)
✓ disc_year    | Discovery Year                      | 100.0% (38,779)

Planets with minimum required data: 18,274 (47.1%)


In [3]:
# Cell 3 - Outlier Detection and Analysis
print("\nOutlier Detection:")
print("="*40)

# Define physically reasonable ranges based on known astronomy
physical_limits = {
    'pl_rade': (0.1, 30),        # From smallest known to largest gas giants
    'pl_bmasse': (0.01, 5000),   # From small asteroids to brown dwarfs  
    'pl_orbsmax': (0.001, 1000), # Very close hot Jupiters to wide binaries
    'st_teff': (1000, 50000),    # From cool brown dwarfs to hot O stars
    'st_mass': (0.08, 100),      # From minimum star mass to massive stars
    'pl_eqt': (1, 3000)          # From very cold to very hot planets
}

# Analyze outliers for each feature
outlier_summary = {}
for feature, (min_val, max_val) in physical_limits.items():
    if feature in df_raw.columns:
        data = df_raw[feature].dropna()
        outliers_low = (data < min_val).sum()
        outliers_high = (data > max_val).sum()
        total_outliers = outliers_low + outliers_high
        outlier_summary[feature] = {
            'total_outliers': total_outliers,
            'percent': (total_outliers / len(data)) * 100,
            'too_low': outliers_low,
            'too_high': outliers_high,
            'actual_range': (data.min(), data.max())
        }
        
        print(f"{feature:12} | Range: {data.min():8.2f} - {data.max():8.2f}")
        print(f"             | Outliers: {total_outliers:,} ({outlier_summary[feature]['percent']:.1f}%)")
        if outliers_high > 0:
            print(f"             | Extreme high values detected")


Outlier Detection:
pl_rade      | Range:     0.27 -  4282.98
             | Outliers: 86 (0.3%)
             | Extreme high values detected
pl_bmasse    | Range:     0.01 - 25426.40
             | Outliers: 180 (2.7%)
             | Extreme high values detected
pl_orbsmax   | Range:     0.00 - 19000.00
             | Outliers: 14 (0.1%)
             | Extreme high values detected
st_teff      | Range:   415.00 - 57000.00
             | Outliers: 5 (0.0%)
             | Extreme high values detected
st_mass      | Range:     0.00 -    23.56
             | Outliers: 58 (0.2%)
pl_eqt       | Range:    34.00 -  4050.00
             | Outliers: 23 (0.1%)
             | Extreme high values detected


In [4]:
# Cell 4 - Data Cleaning Implementation
print("\nImplementing Data Cleaning:")
print("="*40)

# Start with a copy for cleaning
df_clean = df_raw.copy()
print(f"Starting with: {len(df_clean):,} planets")

# Step 1: Remove extreme outliers using physical limits
cleaning_log = []
for feature, (min_val, max_val) in physical_limits.items():
    if feature in df_clean.columns:
        before_count = len(df_clean)
        
        # Remove outliers
        df_clean = df_clean[
            (df_clean[feature].isna()) |  # Keep NaN values
            ((df_clean[feature] >= min_val) & (df_clean[feature] <= max_val))
        ]
        
        removed = before_count - len(df_clean)
        if removed > 0:
            cleaning_log.append(f"Removed {removed:,} planets with unrealistic {feature}")
            print(f"Removed {removed:,} planets with unrealistic {feature}")

print(f"After outlier removal: {len(df_clean):,} planets")

# Step 2: Remove duplicate planet names (keep most recent data)
before_dedup = len(df_clean)
df_clean = df_clean.sort_values('rowupdate', ascending=False).drop_duplicates(subset=['pl_name'], keep='first')
duplicates_removed = before_dedup - len(df_clean)
if duplicates_removed > 0:
    cleaning_log.append(f"Removed {duplicates_removed:,} duplicate planets")
    print(f"Removed {duplicates_removed:,} duplicate planets")

# Step 3: Create a focused dataset with core features
core_columns = [col for col in core_features.keys() if col in df_clean.columns]
additional_useful = ['pl_orbper', 'pl_orbeccen', 'st_rad', 'st_logg', 'pl_insol']
useful_columns = core_columns + [col for col in additional_useful if col in df_clean.columns]

df_focused = df_clean[useful_columns].copy()
print(f"Focused dataset: {len(df_focused):,} planets, {len(df_focused.columns)} features")


Implementing Data Cleaning:
Starting with: 38,779 planets
Removed 86 planets with unrealistic pl_rade
Removed 178 planets with unrealistic pl_bmasse
Removed 10 planets with unrealistic pl_orbsmax
Removed 4 planets with unrealistic st_teff
Removed 45 planets with unrealistic st_mass
Removed 20 planets with unrealistic pl_eqt
After outlier removal: 38,436 planets
Removed 32,541 duplicate planets
Focused dataset: 5,895 planets, 15 features


In [5]:
# Cell 5 - Missing Data Strategy
print("\nMissing Data Analysis:")
print("="*40)

# Calculate missing data percentages for focused dataset
missing_analysis = pd.DataFrame({
    'feature': df_focused.columns,
    'missing_count': df_focused.isnull().sum(),
    'missing_percent': (df_focused.isnull().sum() / len(df_focused) * 100).round(1),
    'available_count': df_focused.count()
})
missing_analysis = missing_analysis.sort_values('missing_percent', ascending=False)

print("Missing data summary for focused dataset:")
for _, row in missing_analysis.iterrows():
    print(f"{row['feature']:15} | {row['missing_count']:6,} missing ({row['missing_percent']:5.1f}%) | {row['available_count']:6,} available")

# Create different datasets based on completeness requirements
datasets = {}

# Dataset 1: Minimal requirements (for maximum sample size)
min_cols = ['pl_name', 'hostname', 'pl_rade', 'pl_orbsmax', 'st_teff']
datasets['minimal'] = df_focused.dropna(subset=min_cols)

# Dataset 2: Standard requirements (good balance)
standard_cols = min_cols + ['st_mass']
datasets['standard'] = df_focused.dropna(subset=standard_cols)

# Dataset 3: Complete requirements (for highest quality analysis)
complete_cols = standard_cols + ['pl_bmasse', 'pl_eqt']
datasets['complete'] = df_focused.dropna(subset=complete_cols)

print(f"\nDataset options:")
for name, dataset in datasets.items():
    print(f"{name:10} | {len(dataset):6,} planets | Features required: {len(dataset.columns)}")


Missing Data Analysis:
Missing data summary for focused dataset:
pl_eqt          |  4,641 missing ( 78.7%) |  1,254 available
pl_orbeccen     |  4,349 missing ( 73.8%) |  1,546 available
pl_bmasse       |  4,001 missing ( 67.9%) |  1,894 available
pl_insol        |  3,633 missing ( 61.6%) |  2,262 available
pl_orbsmax      |  2,808 missing ( 47.6%) |  3,087 available
pl_rade         |  2,738 missing ( 46.4%) |  3,157 available
st_logg         |  2,599 missing ( 44.1%) |  3,296 available
st_mass         |  1,203 missing ( 20.4%) |  4,692 available
st_teff         |  1,014 missing ( 17.2%) |  4,881 available
st_rad          |    978 missing ( 16.6%) |  4,917 available
pl_orbper       |    296 missing (  5.0%) |  5,599 available
pl_name         |      0 missing (  0.0%) |  5,895 available
hostname        |      0 missing (  0.0%) |  5,895 available
disc_year       |      0 missing (  0.0%) |  5,895 available
discoverymethod |      0 missing (  0.0%) |  5,895 available

Dataset options:
m

In [6]:
# Cell 6 - Data Validation and Final Checks
print("\nData Validation:")
print("="*30)

# Use the standard dataset for our main analysis
df_final = datasets['standard'].copy()

# Validate data ranges make sense
validation_checks = {
    'pl_rade': "Planet radius should be positive",
    'pl_orbsmax': "Orbital distance should be positive", 
    'st_teff': "Stellar temperature should be realistic",
    'st_mass': "Stellar mass should be positive"
}

validation_passed = True
for col, description in validation_checks.items():
    if col in df_final.columns:
        invalid = (df_final[col] <= 0).sum()
        if invalid > 0:
            print(f"❌ {description}: {invalid} invalid values found")
            validation_passed = False
        else:
            print(f"✅ {description}: All values valid")

# Check for reasonable value distributions
print(f"\nFinal dataset statistics:")
numeric_cols = df_final.select_dtypes(include=[np.number]).columns
stats_summary = df_final[numeric_cols].describe()
print(stats_summary.round(3))

print(f"\nCleaning Summary:")
print(f"Original dataset: {len(df_raw):,} planets")
print(f"Final dataset: {len(df_final):,} planets ({len(df_final)/len(df_raw)*100:.1f}% retained)")
print(f"Features: {len(df_final.columns)}")


Data Validation:
✅ Planet radius should be positive: All values valid
✅ Orbital distance should be positive: All values valid
✅ Stellar temperature should be realistic: All values valid
✅ Stellar mass should be positive: All values valid

Final dataset statistics:
        pl_rade  pl_bmasse  pl_orbsmax    st_teff   st_mass    pl_eqt  \
count  1729.000    302.000    1729.000   1729.000  1729.000   368.000   
mean      3.451    292.222       0.301   5520.984     0.962   975.853   
std       3.652    613.641       4.386   1001.348     0.221   482.944   
min       0.276      0.290       0.006   2566.000     0.090   183.000   
25%       1.494      8.055       0.052   5191.000     0.841   598.778   
50%       2.290     54.515       0.087   5641.000     0.956   878.500   
75%       3.108    293.118       0.159   5910.000     1.084  1309.250   
max      24.437   4417.837     156.000  27730.000     1.960  2594.300   

       disc_year  pl_orbper  pl_orbeccen    st_rad  st_logg  pl_insol  
coun

In [7]:
# Cell 7 - Save Cleaned Dataset
print("\nSaving Cleaned Data:")
print("="*30)

# Save the cleaned dataset
output_path = '../data/processed/cleaned_data.csv'
df_final.to_csv(output_path, index=False)
print(f"Saved cleaned dataset to: {output_path}")

# Save cleaning log
log_path = '../data/processed/cleaning_log.txt'
with open(log_path, 'w') as f:
    f.write("XO Project - Data Cleaning Log\n")
    f.write("="*40 + "\n\n")
    f.write(f"Original dataset: {len(df_raw):,} planets\n")
    f.write(f"Final dataset: {len(df_final):,} planets\n")
    f.write(f"Retention rate: {len(df_final)/len(df_raw)*100:.1f}%\n\n")
    f.write("Cleaning steps performed:\n")
    for step in cleaning_log:
        f.write(f"- {step}\n")

print(f"Saved cleaning log to: {log_path}")

# Display final dataset preview
print(f"\nFinal cleaned dataset preview:")
print(df_final.head())

print(f"\nDataset ready for feature engineering phase!")
print(f"Proceed to: 03_feature_engineering.ipynb")


Saving Cleaned Data:
Saved cleaned dataset to: ../data/processed/cleaned_data.csv
Saved cleaning log to: ../data/processed/cleaning_log.txt

Final cleaned dataset preview:
          pl_name hostname  pl_rade  pl_bmasse  pl_orbsmax  st_teff  st_mass  \
37029   TOI-880 c  TOI-880    4.950        NaN     0.06350   5050.0     0.87   
37031  TOI-880.02  TOI-880    2.190        NaN     0.03490   5050.0     0.87   
38289   WASP-69 b  WASP-69   11.210      82.58     0.04530   4792.0     0.83   
4786     K2-233 b   K2-233    1.315       2.40     0.03293   4796.0     0.79   
4788     K2-233 c   K2-233    1.272       4.60     0.06640   4796.0     0.79   

       pl_eqt discoverymethod  disc_year  pl_orbper  pl_orbeccen  st_rad  \
37029   805.0         Transit       2025   6.387270          NaN   0.830   
37031  1085.0         Transit       2024   2.575710          NaN   0.830   
38289   971.0         Transit       2014   3.868139          0.0   0.801   
4786   1127.0         Transit       2018  